In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_cbohealth_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100
niter = 1000
optimise_mu = [False, True]
exact = [True, True]
mc_samples = 100

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

/nfs/ghome/live/danceh/.local/mambaforge/envs/causalgp/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45509 instead
  warnings.warn(


In [4]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,
                             seed,
                             n,ntest,
                             optimise_mu = optimise_mu[i], 
                             exact = exact[i],
                             mc_samples = mc_samples)
            futures += [f]

In [5]:
# Check on futures
#futures

In [6]:
# Getting results
results = client.gather(futures)

/nfs/ghome/live/danceh/Causal-GP/causal-KL-GP/src/CBO.py:8: SyntaxWarning: invalid escape sequence '\h'
  """
/nfs/ghome/live/danceh/Causal-GP/causal-KL-GP/src/CBO.py:23: SyntaxWarning: invalid escape sequence '\h'
  """


ValueError: Expected value argument (Tensor of shape (100,)) to be within the support (Real()) of the distribution Normal(loc: 0.0, scale: 1.0), but found invalid values:
tensor([-1.7483, -1.3583, -1.1455, -1.0065, -0.9065, -0.8300, -0.7690, -0.7190,
        -0.6771, -0.6412, -0.6101, -0.5829, -0.5589, -0.5375, -0.5183, -0.5011,
        -0.4855, -0.4714, -0.4586, -0.4469, -0.4363, -0.4267, -0.4179, -0.4099,
        -0.4028, -0.3963, -0.3905, -0.3853, -0.3808, -0.3768, -0.3734, -0.3705,
        -0.3682, -0.3664, -0.3651, -0.3644, -0.3641, -0.3644, -0.3651, -0.3664,
        -0.3682, -0.3705, -0.3734, -0.3768, -0.3808, -0.3853, -0.3905, -0.3963,
        -0.4028, -0.4099, -0.4179, -0.4267, -0.4363, -0.4469, -0.4586, -0.4714,
        -0.4855, -0.5011, -0.5183, -0.5375, -0.5589, -0.5829, -0.6101, -0.6412,
        -0.6771, -0.7190, -0.7690, -0.8300, -0.9065, -1.0065, -1.1455, -1.3583,
        -1.7483, -2.9176,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,    

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "cbohealth_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))